In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 
              'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 
              'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 
              'sqft_lot':int, 'view':int}

In [3]:
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

In [4]:
def get_numpy_data(data, features, output):
    '''This function returns a ‘feature_matrix’ (2D array) consisting of first a column of ones followed by columns containing 
    the values of the input features in the data set in the same order as the input list. It also returns an ‘output_array’ 
    which is an array of the values of the output in the data set (e.g. ‘price’).'''
    data['constant'] = np.ones(data[features[0]].shape)
    features = ['constant'] + features
    feature_matrix = np.array(data[features], dtype=float)
    output_array = np.array(data[output], dtype=float)
    return feature_matrix, output_array

def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return predictions

def normalize_features(features):
    norms = np.linalg.norm(np.array(features), axis=0)
    normalized_features = features / norms
    return normalized_features, norms

 We seek to obtain a sparse set of weights by minimizing the LASSO cost function
 
 SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
 
 (By convention, we do not include w[0] in the L1 penalty term. We never want to push the intercept to zero.)

In [5]:
input_features = ['sqft_living', 'bedrooms']
output_features = ['price']
feature_matrix, output_array = get_numpy_data(sales, input_features, output_features)
normalized_feature_matrix, feature_norms = normalize_features(feature_matrix)

initial_weights = np.array((1,4,1))

max_iter = 10
iterator = 0
strength_lambda = 1e5

weights = initial_weights
all_features = ['constant'] + input_features

In [6]:
ro = []
for j in range(0, len(input_features)+1):
    weights_nj = weights
    weights_nj[j] = 0.
    ro_i = np.dot(normalized_feature_matrix[:,j], 
                output_array - predict_output(normalized_feature_matrix, weights_nj).reshape(output_array.shape))
    ro = ro + [ro_i]

In [7]:
pd.DataFrame(ro, all_features)

,0
constant,7.940030e+07
sqft_living,8.793947e+07
bedrooms,8.096670e+07


Quiz Question: Recall that, whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, the corresponding weight w[i] is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?

The range is [ ro[2] - lambda/2, ro[2] + lambda/2 ] = [ ro[1], ro[2] + abs(ro[1]-ro[2]) ]

In [8]:
print('The range of l1 penalty is [%.3e, %.3e]' % ( 2.*(ro[2]-ro[1]), 2.*np.abs((ro[2]-ro[1]))))

The range of l1 penalty is [-1.395e+07, 1.395e+07]


In [9]:
print('The range of ro given by the above l1 penalty is [%.6e, %.6e]' % (ro[2]-2.*np.abs((ro[2]-ro[1]))/2., ro[2]+2.*np.abs((ro[2]-ro[1]))/2.))

The range of ro given by the above l1 penalty is [7.399394e+07, 8.793947e+07]


In [10]:
def lasso_coordinate_descent_step(j, feature_matrix, output, weights, l1_penalty):
    feature_matrix = np.array(feature_matrix)
#     print(feature_matrix[:,0].shape)
#     print(np.array(output).reshape(feature_matrix[:,0].shape).shape)
#     print(predict_output(feature_matrix, weights).shape)
#     print((weights[j] * feature_matrix[:,j]).shape)
    ro_j = np.dot(feature_matrix[:,j], (output.reshape(feature_matrix[:,0].shape) - predict_output(feature_matrix, weights).reshape(feature_matrix[:,0].shape) + weights[j] * feature_matrix[:,j]).T)
#     print(ro_j.shape)
    if j == 0:
        new_weight_i = ro_i
    elif ro_j < -l1_penalty/2.:
        new_weight_i = ro_j + l1_penalty/2.
    elif ro_j >  l1_penalty/2.:    
        new_weight_i = ro_j - l1_penalty/2.
    else:
        new_weight_i = 0.
    return new_weight_i

In [11]:
# # should print 0.425558846691
# sample_matrix = np.array([[3./math.sqrt(13),1./math.sqrt(10)], [2./math.sqrt(13),3./math.sqrt(10)]])
# sample_output = np.array([1., 1.])
# sample_weights = np.array([1., 4.])
# print(lasso_coordinate_descent_step(1, sample_matrix, sample_output, sample_weights, 0.1))

In [12]:
input_features = ['sqft_living', 'bedrooms']
output_features = ['price']
feature_matrix, output_array = get_numpy_data(sales, input_features, output_features)
normalized_feature_matrix, feature_norms = normalize_features(feature_matrix)
initial_weights = np.array([1., 4., 1.])

print(lasso_coordinate_descent_step(1, normalized_feature_matrix, output_array, initial_weights, 0.1))

87939470.7733


In [13]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = np.array(initial_weights)
    weights_changes = np.ones(initial_weights.shape) * 1e99
    max_iter = 1e4
    iterator = 0
    
    while np.max(weights_changes) > tolerance and iterator < max_iter:
        for i in range(0, weights.size):
            new_weights_i = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            weights_changes[i] = np.abs(new_weights_i - weights[i])
            weights[i] = new_weights_i
        if iterator % 25 == 0:
            print('#%d iter, max weight change is %.5e' % (iterator, np.max(weights_changes)))
        iterator += 1
    
    print('#%d iter, final max weight change is %.5e' % (iterator, np.max(weights_changes)))
    return weights

In [14]:
input_features = ['sqft_living', 'bedrooms']
output_features = ['price']
feature_matrix, output_array = get_numpy_data(sales, input_features, output_features)
normalized_feature_matrix, feature_norms = normalize_features(feature_matrix)

initial_weights = np.array([0., 0., 0.])
l1_penalty = 1e7
tolerance = 1.

optimized_weights = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output_array, initial_weights, l1_penalty, tolerance)


#0 iter, max weight change is 8.09667e+07
#25 iter, max weight change is 2.68623e+04
#50 iter, max weight change is 1.48914e+03
#75 iter, max weight change is 8.25520e+01
#100 iter, max weight change is 4.57635e+00
#115 iter, final max weight change is 9.05822e-01


In [15]:
print('the RSS of the learned model on the normalized dataset is %.3e' 
      % sum((output_array.reshape(np.array(normalized_feature_matrix)[:,0].shape) - predict_output(normalized_feature_matrix, optimized_weights)) ** 2))

the RSS of the learned model on the normalized dataset is 2.665e+15


In [16]:
pd.DataFrame(optimized_weights, all_features)

,0
constant,8.096670e+07
sqft_living,1.282280e+07
bedrooms,-4.185736e+06


### Evaluating LASSO fit with more features


In [17]:
test = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)
train = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)

In [18]:
input_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors','waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement','yr_built', 'yr_renovated']
all_features = ['constant'] + input_features
output_features = ['price']
feature_matrix, output_array = get_numpy_data(train, input_features, output_features)
normalized_feature_matrix, feature_norms = normalize_features(feature_matrix)

In [19]:
initial_weights = np.zeros([len(input_features)+1, 1])
l1_penalty = 1e7
tolerance = 1.

weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output_array, initial_weights, l1_penalty, tolerance)
pd.DataFrame(weights1e7, all_features)

#0 iter, max weight change is 8.09667e+07
#25 iter, max weight change is 5.71037e+05
#50 iter, max weight change is 3.47899e+03
#75 iter, max weight change is 7.48652e+01
#100 iter, max weight change is 1.61096e+00
#105 iter, final max weight change is 8.71635e-01


,0
constant,8.096670e+07
bedrooms,0.000000e+00
bathrooms,0.000000e+00
sqft_living,1.874117e+07
sqft_lot,0.000000e+00
floors,0.000000e+00
waterfront,3.486503e+06
view,8.902526e+06
condition,0.000000e+00
grade,0.000000e+00


In [20]:
initial_weights = np.zeros([len(input_features)+1, 1])
l1_penalty = 1e8
tolerance = 1.

weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output_array, initial_weights, l1_penalty, tolerance)
pd.DataFrame(weights1e8, all_features)

#0 iter, max weight change is 8.09667e+07
#2 iter, final max weight change is 0.00000e+00


,0
constant,8.096670e+07
bedrooms,0.000000e+00
bathrooms,0.000000e+00
sqft_living,0.000000e+00
sqft_lot,0.000000e+00
floors,0.000000e+00
waterfront,0.000000e+00
view,0.000000e+00
condition,0.000000e+00
grade,0.000000e+00


In [21]:
initial_weights = np.zeros([len(input_features)+1, 1])
l1_penalty = 1e4
tolerance = 5e5

weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output_array, initial_weights, l1_penalty, tolerance)
pd.DataFrame(weights1e4, all_features)

#0 iter, max weight change is 8.09667e+07
#25 iter, max weight change is 1.03925e+06
#50 iter, max weight change is 8.09789e+05
#75 iter, max weight change is 6.11379e+05
#100 iter, max weight change is 4.96768e+05
#101 iter, final max weight change is 4.96768e+05


,0
constant,8.096670e+07
bedrooms,-2.066352e+07
bathrooms,1.270318e+07
sqft_living,9.750130e+07
sqft_lot,-1.961819e+06
floors,-3.495361e+06
waterfront,6.497286e+06
view,7.102041e+06
condition,6.708526e+06
grade,1.652119e+07


### Rescaling learned weights

In [22]:
normalized_weights1e7 = weights1e7 / feature_norms.reshape(weights1e7.shape)
normalized_weights1e8 = weights1e8 / feature_norms.reshape(weights1e7.shape)
normalized_weights1e4 = weights1e4 / feature_norms.reshape(weights1e7.shape)

In [23]:
print(normalized_weights1e7[3])

[ 62.47838682]


In [25]:
feature_matrix, output_array = get_numpy_data(test, input_features, output_features)

In [27]:
rss1e7 = sum((output_array - predict_output(feature_matrix, normalized_weights1e7)) ** 2)
rss1e7

array([  3.69143806e+14])

In [28]:
rss1e8 = sum((output_array - predict_output(feature_matrix, normalized_weights1e8)) ** 2)
rss1e8

array([  5.58448276e+14])

In [29]:
rss1e4 = sum((output_array - predict_output(feature_matrix, normalized_weights1e4)) ** 2)
rss1e4

array([  2.26564077e+14])

In [24]:
# while iterator < max_iter:
#     for j in range(0, len(input_features)+1):
#         weights_nj = weights
#         weights_nj[j] = 0.
#         ro = np.dot(normalized_feature_matrix[all_features[j]], 
#                     output_array - predict_output(normalized_feature_matrix, weights_nj).reshape(output_array.shape))
#         if j == 0:
#             weights[j] = ro
#         else:
#             if ro < -strength_lambda/2.:
#                 weights[j] = ro + strength_lambda/2.
#             elif ro > strength_lambda/2.:
#                 weights[j] = ro - strength_lambda/2.  
#             else:
#                 weights[j] = 0.          
#     iterator += 1

# def lasso_coordinate_descent_step(j, normalized_feature_matrix, all_features, output, weights, l1_penalty):
#     weights_nj = weights
#     weights_nj[j] = 0.
#     ro_i = np.dot(normalized_feature_matrix[all_features[j]], output - predict_output(normalized_feature_matrix, weights_nj).reshape(output_array.shape))
    
#     if j == 0:
#         new_weight_i = ro_i
#     elif ro_i < -l1_penalty/2.:
#         new_weight_i = ro + l1_penalty/2.
#     elif ro_i >  l1_penalty/2.:    
#         new_weight_i = ro - l1_penalty/2.
#     else:
#         new_weight_i = 0.
#     return new_weight_i